<a href="https://colab.research.google.com/github/ManojithBhat/4th-Sem-Lab-programs/blob/main/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip -q install --upgrade folium
!apt install libspatialindex-dev
!pip -q install rtree
!pip -q install geopandas
!pip -q install geojson
!pip -q install geemap==0.17.3
!pip -q uninstall tornado -y
!yes | pip install tornado==5.1.0
!pip -q install rasterio
!pip -q install tqdm
!pip -q install eeconvert
!pip install fastapi uvicorn pyngrok pydantic

In [ ]:
# Standard imports
import os
from tqdm.notebook import tqdm
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
from shapely.geometry import box
from fastapi.middleware.cors import CORSMiddleware

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import geemap.eefolium as emap
import folium

# Deep learning libraries
import torch
from torchvision import datasets, models, transforms

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from typing import List, Dict
from pydantic import BaseModel
import time
import asyncio
import logging
from collections import Counter
import uuid

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ee.Authenticate()
ee.Initialize(project="lulc-sde-el")

In [ ]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class CoordinatePoint(BaseModel):
    lat: float
    lng: float

class InputData(BaseModel):
    data: List[CoordinatePoint]
    shape_name: str


In [ ]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.DEBUG
)
logger = logging.getLogger(__name__)

In [ ]:
classes = [
    'AnnualCrop',
    'Forest',
    'HerbaceousVegetation',
    'Highway',
    'Industrial',
    'Pasture',
    'PermanentCrop',
    'Residential',
    'River',
    'SeaLake'
]

# Colors for visualization
colors = {
    'AnnualCrop': 'lightgreen',
    'Forest': 'forestgreen',
    'HerbaceousVegetation': 'yellowgreen',
    'Highway': 'gray',
    'Industrial': 'red',
    'Pasture': 'mediumseagreen',
    'PermanentCrop': 'chartreuse',
    'Residential': 'magenta',
    'River': 'dodgerblue',
    'SeaLake': 'blue'
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model transformation
imagenet_mean, imagenet_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

In [ ]:
MODEL_PATH = './drive/My Drive/Colab Notebooks/models/best_model.pth'

def load_model(model_path: str):
    logger.info(f"Loading model from {model_path}")
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(classes))
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval().to(device)
    logger.info("Model loaded and set to eval mode")
    return model

model = load_model(MODEL_PATH)

In [ ]:
def generate_image(region, product='COPERNICUS/S2',
                   min_date='2019-01-01', max_date='2020-01-01',
                   range_min=0, range_max=2000, cloud_pct=10):
    logger.debug("Starting image generation")
    image = ee.ImageCollection(product)
    image = image.filterBounds(region)
    logger.debug(f"Filtered bounds with region: {region.getInfo()['coordinates']}")
    image = image.filterDate(min_date, max_date)
    image = image.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_pct))
    logger.debug(f"Filtered images between {min_date} and {max_date} with <{cloud_pct}% clouds")
    image = image.median()
    image = image.visualize(bands=['B4', 'B3', 'B2'], min=range_min, max=range_max)
    logger.debug("RGB visualization applied to image")
    clipped = image.clip(region)
    logger.debug("Image clipped to region")
    return clipped

# Export to Drive with poll
def export_image(image, filename, region, folder):
    logger.info(f"Exporting image {filename} to Drive folder {folder}")
    task = ee.batch.Export.image.toDrive(
        image=image,
        driveFolder=folder,
        description=filename,
        fileNamePrefix=filename,
        scale=10,
        region=region.geometry(),
        fileFormat='GeoTIFF',
        crs='EPSG:4326',
        maxPixels=9e6
    )
    task.start()
    while True:
        status = task.status()['state']
        logger.debug(f"Image export task status: {status}")
        if status == 'COMPLETED':
            logger.info("Image export completed")
            break
        if status in ('FAILED', 'CANCELLED'):
            logger.error(f"Image export failed: {task.status()}")
            raise RuntimeError(f"Export task ended with status {status}")
        time.sleep(5)
    return task

# Generate tiles
def generate_tiles(image_file, output_file, area_str, size=64):
    logger.info(f"Generating {size}x{size} tiles for {image_file}")
    raster = rio.open(image_file)
    width, height = raster.shape
    geo_dict = {'id': [], 'geometry': []}
    idx = 0
    for w in range(0, width, size):
        for h in range(0, height, size):
            window = rio.windows.Window(h, w, size, size)
            bbox = rio.windows.bounds(window, raster.transform)
            geom = box(*bbox)
            uid = f"{area_str.lower().replace(' ', '_')}-{idx}"
            geo_dict['id'].append(uid)
            geo_dict['geometry'].append(geom)
            idx += 1
    tiles = gpd.GeoDataFrame(geo_dict, crs='EPSG:4326')
    tiles.to_file(output_file, driver="GeoJSON")
    raster.close()
    logger.info(f"Saved {len(tiles)} tiles to {output_file}")
    return tiles

# Predict crop on one tile
def predict_crop(image_path, shapes, classes, model):
    try:
        with rio.open(image_path) as src:
            out_image, out_transform = rio.mask.mask(src, shapes, crop=True)
        logger.debug("Cropped image for prediction")
        # Remove zero border
        _, xnz, ynz = np.nonzero(out_image)
        cropped = out_image[:, xnz.min():xnz.max(), ynz.min():ynz.max()]
        # Save to temp file
        temp_file = f"temp_{uuid.uuid4().hex}.tif"
        meta = src.meta.copy()
        meta.update({"height": cropped.shape[1],
                     "width": cropped.shape[2],
                     "transform": out_transform})
        with rio.open(temp_file, 'w', **meta) as dest:
            dest.write(cropped)
        logger.debug(f"Written temp file {temp_file}")
        img = Image.open(temp_file)
        inp = transform(img).unsqueeze(0).to(device)
        out = model(inp)
        _, pred = torch.max(out, 1)
        label = classes[pred.item()]
        os.remove(temp_file)
        logger.debug(f"Prediction: {label}")
        return label
    except Exception as e:
        logger.error(f"predict_crop error: {e}")
        raise

@app.post("/predict")
def predict(input_data: InputData):
    try:
        coords = [[p.lng, p.lat] for p in input_data.data]
        polygon = ee.Geometry.Polygon([coords])
        region = ee.FeatureCollection([ee.Feature(polygon)])
        shape_name = input_data.shape_name
        output_dir = "/content/drive/My Drive/google_pic"
        os.makedirs(output_dir, exist_ok=True)

        # Image generation and export
        image = generate_image(region)
        export_image(image, shape_name, region, 'google_pic')
        tif_path = os.path.join(output_dir, f"{shape_name}.tif")

        # Boundary via getInfo
        boundary_geo = polygon.getInfo()['coordinates']
        boundary_file = os.path.join(output_dir, f"{shape_name}_boundary.geojson")
        with open(boundary_file, 'w') as bf:
            json.dump({'type':'Polygon','coordinates': boundary_geo}, bf)
        boundary = gpd.read_file(boundary_file)
        logger.info("Boundary GeoJSON loaded")

        # Tiles
        tiles_file = os.path.join(output_dir, f"{shape_name}.geojson")
        tiles = generate_tiles(tif_path, tiles_file, shape_name)
        tiles = gpd.sjoin(tiles, boundary, predicate='within')
        logger.info(f"Filtered tiles count: {len(tiles)}")

        # Predictions
        labels = []
        for _idx in tqdm(range(len(tiles)), desc="Predicting tiles"):
            labels.append(predict_crop(tif_path, [tiles.geometry.iloc[_idx]], classes, model))
        tiles['pred'] = labels

        # Compute distribution
        tile_counts = Counter(labels)
        total = len(labels)
        class_pct = {cls: (tile_counts.get(cls,0)/total)*100 for cls in classes}
        logger.info(f"Class percentages: {class_pct}")

        # Save preds
        preds_file = os.path.join(output_dir, f"{shape_name}_preds.geojson")
        tiles.to_file(preds_file, driver="GeoJSON")

        # Return payload
        return {
            "status": "success",
            "shape_name": shape_name,
            "class_distribution": {cls: {"count": tile_counts.get(cls,0), "percent": class_pct[cls]} for cls in classes},
            "predictions_file": preds_file
        }
    except Exception as e:
        logger.exception("Error in /predict")
        raise HTTPException(status_code=500, detail=str(e))

@app.on_event("startup")
async def startup_event():
    # Setup ngrok
    # Note: You'll need an ngrok authtoken for this to work
    import nest_asyncio
    ngrok.set_auth_token("<enter the ngrok api here>")

    public_url = ngrok.connect(8000)
    print("FastAPI running on:", public_url)

<ipython-input-29-b5027f421af5>:153: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [313]
INFO:     Waiting for application startup.
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 622, in api_request
    response = urlopen(request, encoded_data, timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 563, in error
    return self._call_chain(*args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  

SystemExit: 3